# Project on Linkedin Job Analaysis

In [8]:
# Selenium and Bs4 Scrapping Module 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests


# Python Modules
import pandas as pd
import numpy as np

#Importing Regex library
import re

#Importing Time Module
import time

In [178]:
# Importing Below driver to do login by Selenium in Guest Mode
import undetected_chromedriver as uc
import pickle

# Set up undetected Chrome driver
options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")
options.add_argument("--disable-infobars")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")

driver = uc.Chrome(options=options)


In [180]:
#Web-Site URL
url = 'https://www.linkedin.com/jobs/search/?currentJobId=3966193243&geoId=102713980&keywords=advertising&location=India&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true2'

In [181]:
#loading the website into the driver
driver.get(url)


#Setting Implicitly wait condition for better page loading 
driver.implicitly_wait(20)



In [169]:

# Declaring the variables to store values while scrapping
job_ids = []
company_name = []
locations = []
designations = []
details_id = []
industry=[]
employees_count=[]
linkedin_followers=[]
involvement = []
level=[]
total_applicants=[]

#counter for details/Job number
detail = 0

In [182]:

# Loop to iterate through pages starting from index 0 up to 20 (total 20 pages) for web scraping
for j in range(0,20,1):

    #Changing url by passing values as 25* j (ev)
    url = f'https://www.linkedin.com/jobs/search/?currentJobId=3966193243&geoId=102713980&keywords=advertising&location=India&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true2&start={25*j}'
    driver.get(url)
    
    time.sleep(np.random.randint(8,10))

    #26 times as only 25 jobs are visible on each page
    for i in range(1,26):

        # we will store Job id when we reach to the second last element of the page making sure that all elenments get loaded before storing
        #job id
        if i == 24:
            li_tags = driver.find_elements(By.CSS_SELECTOR, 'li[data-occludable-job-id]')#25 elment
            for li in li_tags:
                job_id = li.get_attribute('data-occludable-job-id')
                try:
                    job_ids.append(job_id)
                except:
                    job_ids.append(None)
    
            time.sleep(np.random.randint(1,2)) #sleep condition by giving random values 
        
        #storing company id 
        try:
            nam = driver.find_element(By.XPATH,f"/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[1]/div/ul/li[{i}]/div/div/div[1]/div[1]/div[2]/div[2]/span")
            company_ids.append(nam.text)
        except:
            company_ids.append(None)

        #storing location of Job
        try:
            location = driver.find_element(By.XPATH,f"/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[1]/div/ul/li[{i}]/div/div/div[1]/div[1]/div[2]/div[3]/ul/li")
            locations.append(location.text)
        except:
            locations.append(None)

        #storing Designation of Job
        try:
            designation = driver.find_element(By.XPATH,f"/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[1]/div/ul/li[{i}]/div/div/div[1]/div[1]/div[2]/div[1]/a/span[1]/strong")
            designations.append(designation.text)
        except:
            designations.append(None)
  
        #storing Detail id
        try:
            details_id.append(detail)  # number of detail id 
        except:
            details_id.append(None)


        time.sleep(np.random.randint(1,2))

        # Clicking Next child div in the results section
        element = driver.find_element(By.XPATH,f"/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[1]/div/ul/li[{i}]/div/div").click()
        time.sleep(np.random.randint(2,4))

        #Giving Path for scrolling the window
        scroll_div = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[1]/div") 

        # Scroll the Window little bit every 3 seconds
        scroll_pause_time = 3  # time to wait between scrolls in seconds
        scroll_height = 170  # number of pixels to scroll by each time
        if(i%1 == 0):
            for _ in range(1):  # Adjust the range as needed for the number of scrolls
                # Scroll the div down by the specified number of pixels
                driver.execute_script("arguments[0].scrollTop += arguments[1];", scroll_div, scroll_height)
                
                # Wait for the specified pause time
                time.sleep(scroll_pause_time)

        #storing Employee Counts
        try:
            find  = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/section/section/div[1]/div[2]")
        
            employees_count.append(find.text)
         
        except:
            employees_count.append(None)

        #storing industry name
        try:
            find  = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/section/section/div[1]/div[2]")
        
            industry.append(find.text)
         
        except:
            industry.append(None)
          
    
        #storing linkedin Followers of that company
        try:
            find  = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/section/section/div[1]/div[1]/div/div[2]/div[2]")      
            linkedin_followers.append(find.text)
            
        except:
            linkedin_followers.append(None)

        #storing involvement (full-time,part-time,contract etc.)
        try:
            invol  = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/div[1]/div/div[1]/div[1]/div[4]/ul/li[1]/span/span[2]/span/span[1]")      
            involvement.append(invol.text)
            
        except:
            involvement.append(None)

        #storing Job level (Mid -level , Entry level ,Senior Level etc.)
        try:
            lev = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/div[1]/div/div[1]/div[1]/div[4]/ul/li[1]/span/span[3]")      
            level.append(lev.text)
            
        except:
            level.append(None)

        # storing total applicants applying for particular Job
        try:
            total = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div[2]/div/div[2]/div/div[1]/div/div[1]/div/div[1]/div[1]/div[3]/div/span[5]")      
            total_applicants.append(total.text)
            
        except:
            total_applicants.append(None)

        #increment detail id count as per the job number ( 2nd job detail id should be second ) 
        detail+=1
        time.sleep(np.random.randint(3,5))
  
        

### Storing variables into dictionary 

In [110]:
data_dict = {
    "job_ids": job_ids,
    "company_ids": company_ids,
    "locations": locations,
    "designations": designations,
    "details_id": details_id,
    "industry": industry,
    "employees_count": employees_count,
    "linkedin_followers": linkedin_followers,
    "involvement": involvement,
    "level": level,
    "total_applicants": total_applicants
}

In [112]:
# Showing each series length
for key, value in data_dict.items():
    print(f"Length of {key}: {len(value)}")

Length of job_ids: 75
Length of company_ids: 75
Length of locations: 75
Length of designations: 75
Length of details_id: 75
Length of industry: 75
Length of employees_count: 75
Length of linkedin_followers: 75
Length of involvement: 75
Length of level: 75
Length of total_applicants: 75


In [111]:

# Correcting Length Incase we get some extra rows by below logic

min_len = 75 #minimum length of series found above 

# Reduce each list in the dictionary to the minimum length

for key in data_dict:
    data_dict[key] = data_dict[key][:min_len]



### Converting Dictionary to DataFrame

In [114]:
df = pd.DataFrame(data_dict)

In [115]:
# printing NaN values in each series of Dataframe

df.isna().sum()

job_ids                0
company_ids            5
locations              5
designations           5
details_id             0
industry               0
employees_count        0
linkedin_followers     0
involvement            5
level                 36
total_applicants       1
dtype: int64

### showing Dataframe

In [152]:

# Displaying the Dataframe
df

,Unnamed: 0,job_ids,company_ids,locations,designations,details_id,industry,employees_count,linkedin_followers,involvement,level,total_applicants
0,0,3973300317,Wipro,"Pune, Maharashtra, India (On-site)",Business Analyst,0,"IT Services and IT Consulting 10,001+ employee...","IT Services and IT Consulting 10,001+ employee...","9,367,083 followers",Full-time,NaN,Over 100 applicants
1,1,3974757094,Cognizant,"Pune, Maharashtra, India (On-site)",Process Executive - Data,1,"IT Services and IT Consulting 10,001+ employee...","IT Services and IT Consulting 10,001+ employee...","7,473,534 followers",Full-time,Entry level,23 applicants
2,2,3977531665,American Express,"Gurugram, Haryana, India (Hybrid)",Analyst-Data Analytics,2,"Financial Services 10,001+ employees 77,671 on...","Financial Services 10,001+ employees 77,671 on...","2,222,834 followers",Full-time,Entry level,Over 100 applicants
3,3,3976176383,eClerx,"Pune, Maharashtra, India (On-site)",Analyst,3,"Information Technology & Services 10,001+ empl...","Information Technology & Services 10,001+ empl...","321,131 followers",Full-time,Entry level,25 applicants
4,4,3969988271,Marsh McLennan,"Mumbai, Maharashtra, India (Hybrid)",Speacialist - Data Analytics and Insights,4,"Financial Services 10,001+ employees 76,580 on...","Financial Services 10,001+ employees 76,580 on...","626,941 followers",Full-time,Mid-Senior level,Over 100 applicants
...,...,...,...,...,...,...,...,...,...,...,...,...
1060,70,3812848921,Mygate,"Bengaluru, Karnataka, India (On-site)",Content Associate,1046,Advertising Services 51-200 employees 94 on Li...,Advertising Services 51-200 employees 94 on Li...,"2,077 followers",Full-time,Entry level,0 applicants
1061,71,3979031987,Viacom18 Media Private Limited,"Mumbai, Maharashtra, India (On-site)","Senior Executive - Content Planning, JioCinema",1047,IT Services and IT Consulting 51-200 employees...,IT Services and IT Consulting 51-200 employees...,"41,665 followers",Full-time,Executive,5 applicants
1062,72,3978685187,Market Xcel,"Delhi, India (On-site)",Research Analyst,1048,"Advertising Services 5,001-10,000 employees 9,...","Advertising Services 5,001-10,000 employees 9,...","373,084 followers",Full-time,Entry level,0 applicants
1063,73,3974782037,Bombay Softwares,"Navi Mumbai, Maharashtra, India",Bombay Softwares - Executive - Digital Marketing,1049,Space Research and Technology 201-500 employee...,Space Research and Technology 201-500 employee...,"219,324 followers",Full-time,Entry level,70 applicants


### Storing DataFrame in CSV file

In [153]:
final = df_combined.to_csv('final.csv')

# Cleaning

In [309]:
# Reading from CSV File
final = pd.read_csv('final.csv')  
final.head()

,Unnamed: 0.1,Unnamed: 0,job_ids,company_ids,locations,designations,details_id,industry,employees_count,linkedin_followers,involvement,level,total_applicants
0,0,0,3973300317,Wipro,"Pune, Maharashtra, India (On-site)",Business Analyst,0,"IT Services and IT Consulting 10,001+ employee...","IT Services and IT Consulting 10,001+ employee...",9367083,Full-time,NaN,Over 100 applicants
1,1,1,3974757094,Cognizant,"Pune, Maharashtra, India (On-site)",Process Executive - Data,1,"IT Services and IT Consulting 10,001+ employee...","IT Services and IT Consulting 10,001+ employee...",7473534,Full-time,Entry level,23 applicants
2,2,2,3977531665,American Express,"Gurugram, Haryana, India (Hybrid)",Analyst-Data Analytics,2,"Financial Services 10,001+ employees 77,671 on...","Financial Services 10,001+ employees 77,671 on...",2222834,Full-time,Entry level,Over 100 applicants
3,3,3,3976176383,eClerx,"Pune, Maharashtra, India (On-site)",Analyst,3,"Information Technology & Services 10,001+ empl...","Information Technology & Services 10,001+ empl...",321131,Full-time,Entry level,25 applicants
4,4,4,3969988271,Marsh McLennan,"Mumbai, Maharashtra, India (Hybrid)",Speacialist - Data Analytics and Insights,4,"Financial Services 10,001+ employees 76,580 on...","Financial Services 10,001+ employees 76,580 on...",626941,Full-time,Mid-Senior level,Over 100 applicants


In [310]:
# Making Copy
clean = final.copy()
clean.head()



,Unnamed: 0.1,Unnamed: 0,job_ids,company_ids,locations,designations,details_id,industry,employees_count,linkedin_followers,involvement,level,total_applicants
0,0,0,3973300317,Wipro,"Pune, Maharashtra, India (On-site)",Business Analyst,0,"IT Services and IT Consulting 10,001+ employee...","IT Services and IT Consulting 10,001+ employee...",9367083,Full-time,NaN,Over 100 applicants
1,1,1,3974757094,Cognizant,"Pune, Maharashtra, India (On-site)",Process Executive - Data,1,"IT Services and IT Consulting 10,001+ employee...","IT Services and IT Consulting 10,001+ employee...",7473534,Full-time,Entry level,23 applicants
2,2,2,3977531665,American Express,"Gurugram, Haryana, India (Hybrid)",Analyst-Data Analytics,2,"Financial Services 10,001+ employees 77,671 on...","Financial Services 10,001+ employees 77,671 on...",2222834,Full-time,Entry level,Over 100 applicants
3,3,3,3976176383,eClerx,"Pune, Maharashtra, India (On-site)",Analyst,3,"Information Technology & Services 10,001+ empl...","Information Technology & Services 10,001+ empl...",321131,Full-time,Entry level,25 applicants
4,4,4,3969988271,Marsh McLennan,"Mumbai, Maharashtra, India (Hybrid)",Speacialist - Data Analytics and Insights,4,"Financial Services 10,001+ employees 76,580 on...","Financial Services 10,001+ employees 76,580 on...",626941,Full-time,Mid-Senior level,Over 100 applicants


In [385]:
# checking sum of NaN values in Each Series
clean.isna().sum()

Unnamed: 0.1           0
Unnamed: 0             0
job_ids                0
company_ids            0
designations           0
details_id             0
industry               0
employees_count        0
linkedin_followers     0
involvement            0
level                  0
total_applicants       0
City                   0
State                 54
dtype: int64

In [312]:
# Checking Datatypes of Each column 
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052 entries, 0 to 1051
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0.1        1052 non-null   int64 
 1   Unnamed: 0          1052 non-null   int64 
 2   job_ids             1052 non-null   int64 
 3   company_ids         1052 non-null   object
 4   locations           1052 non-null   object
 5   designations        1052 non-null   object
 6   details_id          1052 non-null   int64 
 7   industry            1032 non-null   object
 8   employees_count     1032 non-null   object
 9   linkedin_followers  1052 non-null   int64 
 10  involvement         978 non-null    object
 11  level               713 non-null    object
 12  total_applicants    1046 non-null   object
dtypes: int64(5), object(8)
memory usage: 107.0+ KB


In [313]:
# cleaning all rows where company_ids values are NaN
clean = clean.dropna(subset=['company_ids'])

# cleaning Linkedin_followers column

clean['linkedin_followers'] = clean['linkedin_followers'].str.replace('followers','') #replacing followers to empty space
clean['linkedin_followers'] = clean['linkedin_followers'].str.replace(',','')  #replacing comma to empty space
clean['linkedin_followers'] = clean['linkedin_followers'].fillna(0)  # replacing Nan values with 0
clean['linkedin_followers'] = clean['linkedin_followers'].astype(int)  #int conversion
clean.head(30) # Reading 30 values from DataFrame



,Unnamed: 0.1,Unnamed: 0,job_ids,company_ids,locations,designations,details_id,industry,employees_count,linkedin_followers,involvement,level,total_applicants
0,0,0,3973300317,Wipro,"Pune, Maharashtra, India (On-site)",Business Analyst,0,"IT Services and IT Consulting 10,001+ employee...","IT Services and IT Consulting 10,001+ employee...",9367083,Full-time,NaN,Over 100 applicants
1,1,1,3974757094,Cognizant,"Pune, Maharashtra, India (On-site)",Process Executive - Data,1,"IT Services and IT Consulting 10,001+ employee...","IT Services and IT Consulting 10,001+ employee...",7473534,Full-time,Entry level,23 applicants
2,2,2,3977531665,American Express,"Gurugram, Haryana, India (Hybrid)",Analyst-Data Analytics,2,"Financial Services 10,001+ employees 77,671 on...","Financial Services 10,001+ employees 77,671 on...",2222834,Full-time,Entry level,Over 100 applicants
3,3,3,3976176383,eClerx,"Pune, Maharashtra, India (On-site)",Analyst,3,"Information Technology & Services 10,001+ empl...","Information Technology & Services 10,001+ empl...",321131,Full-time,Entry level,25 applicants
4,4,4,3969988271,Marsh McLennan,"Mumbai, Maharashtra, India (Hybrid)",Speacialist - Data Analytics and Insights,4,"Financial Services 10,001+ employees 76,580 on...","Financial Services 10,001+ employees 76,580 on...",626941,Full-time,Mid-Senior level,Over 100 applicants
5,5,5,3973937959,Sakshath Technologies®,"Mumbai, Maharashtra, India (On-site)",Business Analyst,5,IT Services and IT Consulting 51-200 employees...,IT Services and IT Consulting 51-200 employees...,5631,Full-time,Entry level,94 applicants
6,6,6,3973940307,Michael Page,"Gurgaon, Haryana, India (On-site)",Tech Business Analyst,6,"Staffing and Recruiting 5,001-10,000 employees...","Staffing and Recruiting 5,001-10,000 employees...",3668722,Full-time,Entry level,80 applicants
7,7,7,3974265125,Infosys BPM,"Pune, Maharashtra, India (On-site)",Process Executive,7,"IT Services and IT Consulting 10,001+ employee...","IT Services and IT Consulting 10,001+ employee...",1539122,Full-time,Entry level,Over 100 applicants
8,8,8,3976167209,NetApp,"Bengaluru, Karnataka, India","Business Data Analyst (Power BI, Tableau, Data...",8,"Software Development 10,001+ employees 12,416 ...","Software Development 10,001+ employees 12,416 ...",684025,NaN,NaN,Over 100 applicants
9,9,9,3978493692,Niveus Solutions,"Gurgaon, Haryana, India (On-site)",Associate - Business Analyst,9,"IT Services and IT Consulting 501-1,000 employ...","IT Services and IT Consulting 501-1,000 employ...",33742,Full-time,Entry level,0 applicants


In [315]:
# Printing Shape of Dataframe
clean.shape

(1052, 13)

### temporary Storing the dataframe

In [391]:
hi = clean  # hi backup variable of clean in case of Emergency purpose ( BUY ME A COFFEE)
clean = hi.copy() 
hi.head()


,Unnamed: 0.1,Unnamed: 0,job_ids,company_ids,designations,details_id,industry,employees_count,linkedin_followers,involvement,level,total_applicants,City,State
0,0,0,3973300317,Wipro,Business Analyst,0,IT Services and IT Consulting,10001,9367083,Full-time,Full-time,195,Pune,Maharashtra
1,1,1,3974757094,Cognizant,Process Executive - Data,1,IT Services and IT Consulting,10001,7473534,Full-time,Entry level,23,Pune,Maharashtra
2,2,2,3977531665,American Express,Analyst-Data Analytics,2,Financial Services,10001,2222834,Full-time,Entry level,102,Gurugram,Haryana
3,3,3,3976176383,eClerx,Analyst,3,Information Technology & Services,10001,321131,Full-time,Entry level,25,Pune,Maharashtra
4,4,4,3969988271,Marsh McLennan,Speacialist - Data Analytics and Insights,4,Financial Services,10001,626941,Full-time,Mid-Senior level,165,Mumbai,Maharashtra


In [384]:
# Function to extract service type and number of employees
def extract_info(industry_str):
    if pd.isna(industry_str):
        return None, None
    
    # Extract service type
    service_type = re.findall(r'^[^\d]+', industry_str)
    service_type = service_type[0].strip() if service_type else None
    
    # Extract number of employees
    num_employees = re.findall(r'(\d{1,3}(?:,\d{3})*\+?) employees', industry_str)
    num_employees = num_employees[0].replace(',', '') if num_employees else None
    
    return service_type, num_employees

# Apply the function to the DataFrame
clean[['industry', 'employees_count']] = clean['industry'].apply(lambda x: pd.Series(extract_info(x)))

# Handle missing values and convert numeric columns to integers
clean['employees_count'] = clean['employees_count'].astype(str)
clean['employees_count'] = clean['employees_count'].str.replace('+','')
clean['employees_count']=  clean['employees_count'].str.replace('None','100')
clean['employees_count']=clean['employees_count'].fillna('100')
clean['employees_count']=clean['employees_count'].astype(int)



# Filtering total applicants

clean['total_applicants']=clean['total_applicants'].fillna(0)
clean['total_applicants']=clean['total_applicants'].str.replace('Over','')
clean['total_applicants']=clean['total_applicants'].str.replace('applicants','')
clean['total_applicants']=clean['total_applicants'].str.replace('applicant','')
clean['total_applicants']=clean['total_applicants'].fillna(0)
clean['total_applicants']=clean['total_applicants'].astype(int)

# Defining the function for changing constant 100 appplicants by replacing it with random integers between 101 to 200
def replacer(x):
    if x>=100:
        return np.random.randint(101,200)
    else:
        return x


clean['total_applicants'] = clean['total_applicants'].apply(replacer)


#Cleaning involvment
clean['involvement'] = clean['involvement'].fillna('Full-time') # filling NaN values to full-time as per HAMZA theory


#cleaning level
clean['level'] = clean['level'].str.replace('\nMatches your job preferences, job type is Full-time.','')
clean['level'] = clean['level'].str.replace('\nMatches your job preferences, job type is Contract.','')
clean['level'] = clean['level'].fillna('Full-time')


# AS PER ANKUR SIR GOOGLE SEARCH THOERY YOU WILL GET IT -JOKE

# print(rows_with_nan_state)
clean.locations.unique() # for seeing Unique values in location column 

# Extracting City,State and Hybrid Modes from Locations Column
clean[['City','State','Hybrid']] = clean['locations'].str.split(',',expand =True ) 
clean['State'] = clean['State'].str.replace('India (Hybrid)','')
clean['State'] = clean['State'].str.replace('India (On-site)','')
clean['State'] = clean['State'].str.replace('India','')

clean = clean.drop(['Hybrid']) # drop Hybrid Column as we dont wan't - BY HAMZA THEORY ( JUST A JOKE BY ZOLOO )

# printing Level counts 
clean.level.value_counts()

level
Full-time           376
Entry level         248
Mid-Senior level    207
Associate           159
Internship           44
Executive            12
Director              5
Contract              1
Name: count, dtype: int64

## Storing Cleaned Dataframe into CSV

In [400]:

clean.to_csv('final3.csv')

In [395]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 998 entries, 0 to 1051
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0.1        998 non-null    int64 
 1   Unnamed: 0          998 non-null    int64 
 2   job_ids             998 non-null    int64 
 3   company_ids         998 non-null    object
 4   designations        998 non-null    object
 5   details_id          998 non-null    int64 
 6   industry            998 non-null    object
 7   employees_count     998 non-null    int32 
 8   linkedin_followers  998 non-null    int64 
 9   involvement         998 non-null    object
 10  level               998 non-null    object
 11  total_applicants    998 non-null    int64 
 12  City                998 non-null    object
 13  State               998 non-null    object
dtypes: int32(1), int64(6), object(7)
memory usage: 113.1+ KB
